This library is old (2023), so you will need to downgrade the versions of openai and numpy. 

Therefore, we recommend creating a separate virtual environment.

(Please use requirements_fs.txt)

In [ ]:
# please read original reference : https://github.com/shmsw25/FActScore
%pip install --upgrade factscore

Please change factscore/atomic_facts.py.
line 26 : "InstructGPT" -> "ChatGPT"

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
%pip install ipywidgets

In [ ]:
import nltk
nltk.download('punkt_tab')


In [ ]:
%pip install openai==0.27.8


In [ ]:
!python -m factscore.download_data 

In [ ]:
%pip install torch==2.1.2

In [ ]:
%pip install "numpy<2.0"


In [ ]:
import json
with open("../dataset/rag_truth_test.json", "r") as f:
    data = json.load(f)

In [3]:
data[0]

{'ref': 'The FBI charged a Philadelphia woman on Thursday with trying to travel overseas to fight for ISIS. She\'s one of three women arrested this week on terror charges. Two New York women were also taken into custody. An FBI complaint cites numerous social media messages dating back to August 2013 that were sent by Keonna Thomas, 30, also known as "Young Lioness" and "Fatayat Al Khilafah." One Twitter message said, "If we truly knew the realities ... we all would be rushing to join our brothers in the front lines pray ALLAH accept us as shuhada [martyrs]." Another said, "When you\'re a mujahid [violent jihadi fighter] your death becomes a wedding." The FBI said Thomas purchased an electronic visa to Turkey on March 23. Turkey is known as the easiest place from which to enter Syria and join ISIS. An ISIS manual advises recruits to buy round-trip tickets to vacation spots such as Spain and then purchase tickets for their real destination once they arrive overseas, the FBI said. On Mar

In [ ]:
import json

# make database to retrieve
with open(".cache/factscore/ragtruth_test_db.jsonl", "w", encoding="utf-8") as f:
    for i,d in enumerate(data):
        json_line = {"title": str(i), "text": d["ref"]}
        f.write(json.dumps(json_line, ensure_ascii=False) + "\n")




In [ ]:
from factscore.factscorer import FactScorer

fs = FactScorer(openai_key = ".env")

# once DB file is created, you can reuse it by only specifying `db_path`
fs.register_knowledge_source("ragtruth_test_db")

In [ ]:
# key : index, value : input_text of RAGTruth
topics = ["0"]
generations = [data[0]["text"]]

results = fs.get_score(
    topics = topics,
    generations = generations,
    knowledge_source= "ragtruth_test_db"
)

print(results)

{'score': 1.0, 'respond_ratio': 1.0, 'decisions': [[{'atom': 'The FBI has charged a Philadelphia woman.', 'is_supported': True}, {'atom': "The Philadelphia woman's name is Keonna Thomas.", 'is_supported': True}, {'atom': 'Keonna Thomas was charged with trying to travel overseas.', 'is_supported': True}, {'atom': 'Keonna Thomas was charged with trying to travel overseas to fight for ISIS.', 'is_supported': True}, {'atom': 'Two New York women were arrested.', 'is_supported': True}, {'atom': 'The names of the two women are Noelle Velentzas and Asia Siddiqui.', 'is_supported': True}, {'atom': 'Noelle Velentzas and Asia Siddiqui were accused of planning to build an explosive device.', 'is_supported': True}, {'atom': 'They were accused of planning to build an explosive device for attacks in the United States.', 'is_supported': True}, {'atom': 'The arrests of the two women followed this incident.', 'is_supported': True}, {'atom': 'Thomas purchased an electronic visa to Turkey.', 'is_supported

In [ ]:
from tqdm import tqdm

outputs = []
for i,d in enumerate(tqdm(data)):
    topics = [str(i)]
    generations = [d["text"]]
    results = fs.get_score(topics=topics,
                       generations=generations,
                       knowledge_source="ragtruth_test_db")
    outputs.append(results["score"])

In [ ]:
answers = [d["labels"] for d in data]

In [ ]:
outputs_l = []
for o in outputs:
    if o < 1.0:
        outputs_l.append(1)
    else: # faithful
        outputs_l.append(0)
        
count = Counter(outputs_l)

print(count)

    

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

accuracy_score(answers, outputs_l), f1_score(answers, outputs_l), precision_score(answers, outputs_l), recall_score(answers, outputs_l)

In [ ]:
import os

if os.path.exists("../results_test.json"):
    with open("../results_test.json", "r") as f:
        j_data = json.load(f)
else:
    j_data = []
    for i, d in enumerate(data):
        j_data.append({"id": i, "label": d["labels"], "task": d["task_type"]})

len(j_data)

In [ ]:
for i in range(len(j_data)):
    j_data[i]["fs_score"] = outputs[i]
    j_data[i]["fs_label"] = outputs_l[i]

In [ ]:
j_data[0]

In [ ]:
with open("../results_test.json", "w") as f:
    json.dump(data, f)